In [2]:
import os
# from chromadb.config import Settings
from youtube_transcript_api import YouTubeTranscriptApi 

In [4]:


# assigning srt variable with the list 
# of dictionaries obtained by the get_transcript() function
srt = YouTubeTranscriptApi.get_transcript("iLY_5DMndWY")

with open("subtitle.txt", "a") as file:
    for i in srt:
        file.write(i['text'])


In [50]:
# # Define the chroma settings
# CHROMA_SETTINGS = Settings(
#     chroma_db_impl = 'duckdb+parquet',
#     persist_directory = 'db',
#     anonymized_telemetry = False,
# )

In [11]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings 
from langchain.vectorstores import Chroma
import os
# from constants import CHROMA_SETTINGS


In [7]:
persist_directory = 'db'

In [23]:
%pwd

'd:\\Python Hub\\GenAI\\Youtube Chatbot\\Chatbot-on-YouTube\\research'

In [24]:
os.chdir("../")

In [25]:
%pwd

'd:\\Python Hub\\GenAI\\Youtube Chatbot\\Chatbot-on-YouTube'

In [32]:
def main():
    for root, dirs, files in os.walk('docs'):
        for file in files:
            if file.endswith(".txt"):
                print(file)
                loader = TextLoader(os.path.join(root,file))
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    texts = text_splitter.split_documents(documents)
    
    # Create embeddings
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    # Create vector store
    db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)
    
    db.persist()
    db=None
    

In [33]:
main()

subtitle.txt


d:\Python Hub\GenAI\Youtube Chatbot\Chatbot-on-YouTube\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Python Hub\GenAI\Youtube Chatbot\Chatbot-on-YouTube\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Python Hub\GenAI\Youtube Chatbot\Chatbot-on-YouTube\myenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [34]:
# Create embeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Python Hub\GenAI\Youtube Chatbot\Chatbot-on-YouTube\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [35]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)

In [36]:
retriever = vectordb.as_retriever()

In [37]:
query = "How NIRMA Changed Indian Detergent Market".lower()

In [48]:
query

'how nirma changed indian detergent market'

In [39]:
docs = retriever.get_relevant_documents(query)

d:\Python Hub\GenAI\Youtube Chatbot\Chatbot-on-YouTube\myenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [40]:
docs

[Document(page_content='he Nam named the detergent thathe formulated nirma and he would go onand make this a global brand he also puta beautiful illustration of a girl inthe white frog on the front of thepacket which was symbolic of her thiswas the beginning of the legendarywashing powder nirma now cenm decided totake his detergent to the market sowhile all the other prominent detergentBrands like Hindustan liver surf wasselling at 13.5 rupees per kg ksen bystarted selling his deterg nma at just3.5 rupees per kg this', metadata={'source': 'docs\\subtitle.txt'}),
 Document(page_content="but 1988 nirma hadcaptured 60% of the detergent market andhad overtaken the mainstream detergentserf just like the jingle claimed nirmahad become suban nirma and kasen'sMission to immortalize his daughter wasalso successful as of today nma is oneof the largest producers of sodash whenit comes to volumes along with this nIrma has also Diversified into manyother Industries like personal careproducts health

## Model

In [41]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64
import textwrap
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain. vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA



In [42]:
checkpoint = "MBZUAI/LaMini-T5-738M"

In [43]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [44]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    device_map="auto",
    torch_dtype=torch.float32,
    offload_folder="offload"

)

In [45]:
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model=model,
        tokenizer=tokenizer,
        max_length=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.95

    )

    local_llm=HuggingFacePipeline(pipeline=pipe)
    return local_llm

In [46]:
def qa_llm():
    llm=llm_pipeline()
    embeddings=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db=Chroma(persist_directory="db",embedding_function=embeddings)
    retriever=db.as_retriever()
    qa=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )

    return qa

In [47]:
def process_answer(instruction):
    response=''
    instruction=instruction
    qa = qa_llm()
    generated_text=qa(instruction)
    answer=generated_text['result']
    return answer, generated_text


In [49]:
process_answer("how nirma changed indian detergent market")

d:\Python Hub\GenAI\Youtube Chatbot\Chatbot-on-YouTube\myenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(
d:\Python Hub\GenAI\Youtube Chatbot\Chatbot-on-YouTube\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Python Hub\GenAI\Youtube Chatbot\Chatbot-on-YouTube\myenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: 

('Nirma changed the Indian detergent market by introducing a unique price point of NMA detergent that was cheaper than other detergent brands.',
 {'query': 'how nirma changed indian detergent market',
  'result': 'Nirma changed the Indian detergent market by introducing a unique price point of NMA detergent that was cheaper than other detergent brands.',
  'source_documents': [Document(page_content='he Nam named the detergent thathe formulated nirma and he would go onand make this a global brand he also puta beautiful illustration of a girl inthe white frog on the front of thepacket which was symbolic of her thiswas the beginning of the legendarywashing powder nirma now cenm decided totake his detergent to the market sowhile all the other prominent detergentBrands like Hindustan liver surf wasselling at 13.5 rupees per kg ksen bystarted selling his deterg nma at just3.5 rupees per kg this', metadata={'source': 'docs\\subtitle.txt'}),
   Document(page_content="but 1988 nirma hadcaptured